In [23]:
#Mutation based fuzzer


import random
import string

#config
MAX_NUMBER_OF_MUTATORS = 20
INSERT_MAX_LENGTH = 30
DELETE_MAX_LENGTH = 20

ALL_ASCII = []
for i in range(0, 128):
    ALL_ASCII.append(chr(i))

def mutator_insert(input_data):
    length = random.randrange(5, INSERT_MAX_LENGTH)
    position = random.randrange(0, len(input_data))
    random_bytes = bytearray()
    #random_bytes = ""
    random_bytes += bytes([random.randrange(0, 255)])
    return input_data[:position] + random_bytes + input_data[position:]

def mutator_delete(input_data):
    length = len(input_data)
    position = random.randrange(5, length)
    count = random.randrange(0, min(length, DELETE_MAX_LENGTH))
    return input_data[:position] + input_data[position+count:]
        
active_mutators=[mutator_insert, mutator_delete]
#active_mutators=[mutator_insert]

def mutate_corpus(corpus,run_id):
    random.seed(run_id)
    active_corpus=random.choice(corpus)
    return mutate(active_corpus, run_id)

def mutate(input_data, run_id):
    random.seed(run_id)
    number_of_mutations = random.randint(0, MAX_NUMBER_OF_MUTATORS)
    for i in range(0, number_of_mutations):
        mutator = random.choice(active_mutators)
        input_data = mutator(input_data)
    return input_data

    

In [24]:
# Corpus
#mutate(bytes(10), 1)
#import glob
#print(glob.glob("./corpus/*"))
#corpus_files = glob.glob("./corpus/*")
#corpus=[]

#for file in corpus_files:
#    with open(file, "rb") as f:
#        corpus.append(f.read())

#print(mutate_corpus(corpus,1))

corpus = [b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA']

In [25]:
import subprocess

def runner(testcase):
    print(f"Executing test case - {testcase}%")
    with open("testcase.bin", "wb") as f:
        f.write(testcase)
        f.close()
    result = subprocess.run(["./mqtt", "testcase.bin"], shell=False, timeout=1, env={"LSAN_OPTIONS" : "detect_leaks=0"})
    return result.returncode

#def runner(testcase):
#    result = subprocess.run(["./coverage", testcase], shell=False, timeout =1)
#    return result.returncode

In [26]:
#runner(b"asdffffhgkhgjhghjg")

In [28]:
!rm mqtt.c.gcov
!rm mqtt_pal.gcno
!rm mqtt.gcno
#!gcc -g -fsanitize=address mqtt-fuzzer.c -Iinclude src/* -o mqtt-fuzzer_gcc
#!gcc --coverage coverage.c -o coverage
#!clang -g -O0 -fsanitize=address mqtt-fuzzer.c -Iinclude src/* -o mqtt-fuzzer
#!clang -g -fsanitize=address mqtt-fuzzer.c -Iinclude src/* -o mqtt-fuzzer
!gcc -g --coverage -fsanitize=address -Iinclude src/* -o mqtt

# #Dry run to determine base coverage
for c in corpus:
    runner(c)

def get_coverage():
    r = subprocess.run(["gcov", "mqtt"], stdout=subprocess.PIPE)
    return int(float(r.stdout.decode('utf-8').split('Lines executed:')[1].split('% of')[0]))
    
run_id =0 
current_coverage=get_coverage()
print(f"Base Coverage is {current_coverage}%")
while True:
    testcase = mutate_corpus(corpus, run_id)
    result=runner(testcase)

    coverage =get_coverage()
    
    #check if coverage increased
    if(coverage > current_coverage):
        current_coverage = coverage
        corpus.append(testcase)
        print("Current coverage - " + str(current_coverage) + "%")
    



# while True:
#     testcase = mutate_corpus(corpus, run_id)
#     result = runner(testcase)
#     #print(result)
#     #print(f"resultcode - {result}")
#     #if result is not None:
#     if result != 0:
#         print(f"Sucess : {run_id}")
#     run_id +=1

Executing test case - b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'%
Base Coverage is 5%


ValueError: empty range for randrange() (5, 5, 0)

In [13]:
testcase = mutate_corpus(corpus, 52)
result = runner(testcase)
print(testcase)
print(result)

b'<\x11(+{;\xa0AAAAAAA\xda\xf1^hp\xa1\x08\xeb#(AA'
1
